# Pipeline de tratamiento de datos

In [ ]:
# Librerias
import pandas as pd 
import numpy as np 
import datetime 
import os 
import re 

In [ ]:
# Lectura de archivos 
files = {
    "categoria": "data/categoria.csv",
    "cliente": "data/cliente.csv",
    "events": "data/events.csv",
    "marca": "data/marca.csv",
    "producto": "data/producto.csv",
}

def load_csv_robusto(path):
    for enc in ["utf-8", "latin-1", "cp1252"]:
        try:
            return pd.read_csv(path, encoding=enc, low_memory=False)
        except Exception:
            continue
    # último intento sin encoding explícito
    return pd.read_csv(path, low_memory=False)

# --- Parsear fechas m/d/yy (pivot: <30 -> 2000s; >=30 -> 1900s) ---
def parse_mdY(s, pivot=30):
    if pd.isna(s): 
        return pd.NaT
    m = re.match(r"\s*(\d{1,2})/(\d{1,2})/(\d{2,4})\s*$", str(s))
    if not m:
        return pd.NaT
    mm, dd, yy = map(int, m.groups())
    if yy < 100:
        yy = 2000 + yy if yy < pivot else 1900 + yy
    try:
        return pd.Timestamp(year=yy, month=mm, day=dd)
    except Exception:
        return pd.NaT

In [ ]:
categoria = load_csv_robusto(files["categoria"])
cliente = load_csv_robusto(files["cliente"])
events = load_csv_robusto(files["events"])
marca = load_csv_robusto(files["marca"])
producto = load_csv_robusto(files["producto"])

### Limpieza categoria

In [3]:
categoria.head()

,id,categoria
0,1,SCOTCH WHISKIES
1,2,STRAIGHT BOURBON WHISKIES
2,3,BLENDED WHISKIES
3,4,IMPORTED DRY GINS
4,5,DECANTERS & SPECIALTY PACKAGES


In [4]:
categoria.duplicated().sum()

0

In [5]:
categoria.isnull().sum()    

id           0
categoria    0
dtype: int64

In [6]:
categoria["categoria"] = categoria["categoria"].str.strip().str.upper()

In [7]:
categoria.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         101 non-null    int64 
 1   categoria  101 non-null    object
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


### Marca

In [8]:
marca.head()

,id,marca
0,1,Diageo Americas
1,2,Heaven Hill Brands
2,3,"Sazerac Co., Inc."
3,4,Sage Beverages
4,5,MHW Ltd


In [9]:
marca.isnull().sum()

id       0
marca    0
dtype: int64

In [10]:
marca.duplicated().sum()

0

In [11]:
marca["marca"] = marca["marca"].str.strip().str.upper()
marca.head()

,id,marca
0,1,DIAGEO AMERICAS
1,2,HEAVEN HILL BRANDS
2,3,"SAZERAC CO., INC."
3,4,SAGE BEVERAGES
4,5,MHW LTD


In [12]:
marca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307 entries, 0 to 306
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      307 non-null    int64 
 1   marca   307 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.9+ KB


### Producto

In [13]:
producto.head()

,id,categoria_id,nombre,marca_id,volumen,precio
0,356475,9.0,Crown Royal Honey,1.0,750,22.49
1,15335,9.0,Crown Royal Regal Apple Mini,1.0,300,11.03
2,81345,9.0,Crown Royal Regal Apple,1.0,200,7.08
3,150318,9.0,Crown Royal Xr Canadian Whiskey,1.0,750,98.99
4,310791,9.0,Crown Royal Canadian Whisky Mini,1.0,300,11.03


In [14]:
producto.duplicated().sum()

0

In [15]:
producto.describe()

,id,categoria_id,marca_id,volumen,precio
count,12026.000000,10998.000000,11150.000000,12026.000000,12020.000000
mean,233928.299850,23.148663,41.812735,1033.562448,38.770502
std,133810.910123,20.469181,62.779059,4717.072506,309.980074
min,15.000000,1.000000,1.000000,0.000000,0.000000
25%,117674.750000,7.000000,7.000000,750.000000,8.250000
50%,234303.500000,18.000000,19.000000,750.000000,14.250000
75%,349531.750000,37.000000,39.000000,750.000000,25.500000
max,466861.000000,101.000000,307.000000,225000.000000,9702.000000


In [16]:
producto.isnull().sum()

id                 0
categoria_id    1028
nombre             0
marca_id         876
volumen            0
precio             6
dtype: int64

Se ha verificado que existen datos en volumen y en precio que son igual a cero; deben ser corregidos

In [17]:
producto[producto["volumen"]==0]

,id,categoria_id,nombre,marca_id,volumen,precio
11142,198041,NaN,Item Placeholder for unlisted items,304.0,0,1.50
11143,21551,NaN,Test,304.0,0,5.82


In [18]:
producto[producto["precio"]==0]

,id,categoria_id,nombre,marca_id,volumen,precio
8094,116587,19.0,Turi Vodka Estonian Dno,47.0,750,0.0
8104,207115,19.0,Mezzaluna Vodka,47.0,750,0.0
8798,357802,11.0,Littlemill 12yr,63.0,750,0.0
9940,51360,19.0,Hammer Sickle Vodka,112.0,750,0.0
10090,310556,24.0,Voo Doo Spice Rum,139.0,750,0.0


Los volumen cero se eliminaran y para el precio 0 se les asignara la media del precio de la categoria y volumen que tienen registrado, esta ultima accion podra ser ejecutada hasta que se corrijan los problemas de nulos de las columnas

In [19]:
producto = producto[producto["volumen"]!=0]

In [20]:
producto[producto["precio"].isnull()]

,id,categoria_id,nombre,marca_id,volumen,precio
1146,218626,28.0,Dr. Mcgillicuddy's Cherry Schnapps,3.0,375,NaN
4178,281544,36.0,E & J Cask & Cream (dno),18.0,375,NaN
6144,17189,11.0,Glen Keith Sing Malt Scotch(dno),7.0,750,NaN
6442,151538,8.0,Barton Light Rum,19.0,1750,NaN
7864,51354,19.0,Boru Crazzberry Vodka,40.0,750,NaN
10983,189604,55.0,Brandy Stock 191.2,250.0,15000,NaN


In [21]:
producto[(producto["categoria_id"]==28) & ( producto["volumen"]==375)]["precio"].median()

5.31

Los precios faltantes se les asignará la moda del precio de la categoria a la que pertenece, con el mismo volumen.

In [22]:
producto["precio"] = producto["precio"].fillna(
    producto.groupby(["categoria_id", "volumen"])["precio"].transform("median")
)

In [23]:
producto[producto["marca_id"].isnull() & producto["categoria_id"].isnull()]["nombre"].unique()

array(['Generic Drink'], dtype=object)

Debido a que todos los casos de marca_id null coinciden con categoria_id null, y el nombre es el mismo en todos los registros, se unificara la categoria como "OTHER" y la marca "OTHER"

In [24]:
marca = pd.concat([marca, pd.DataFrame({"id": [308], "marca": ["OTHER"]})], ignore_index=True)
marca.tail()

,id,marca
303,304,IOWA ABD
304,305,PARK STREET-G'DAY
305,306,ROYAL WINE CORPORATION
306,307,PURPLE VALLEY IMPORTS
307,308,OTHER


In [25]:
categoria = pd.concat([categoria, pd.DataFrame({"id": [102], "categoria": ["OTHER"]})], ignore_index=True)
categoria.tail()

,id,categoria
97,98,AMARETTO - IMPORTED
98,99,HIGH PROOF BEER - AMERICAN
99,100,SPECIALTY ITEMS - SPIRITS - AME
100,101,IMPORTED VODKA - CHERRY
101,102,OTHER


In [26]:
# Asignacion de valores en producto marca_id y categoria_id nulos
producto.loc[
    producto["marca_id"].isnull() & producto["categoria_id"].isnull(),
    ["marca_id", "categoria_id"]
] = [308, 102]

In [27]:
producto[producto["nombre"]=="Generic Drink"].isnull().sum() # Verificacion de la correccion

id              0
categoria_id    0
nombre          0
marca_id        0
volumen         0
precio          0
dtype: int64

In [28]:
producto[producto["categoria_id"].isnull()]

,id,categoria_id,nombre,marca_id,volumen,precio
2221,175668,NaN,Evan Williams Egg Nog,2.0,750,6.69
2830,396750,NaN,Hudson New York Corn,10.0,375,25.64
2955,260901,NaN,Svedka Stars & Stripes,12.0,1750,22.11
2956,43831,NaN,Svedka Cucumber Lime Mini,12.0,600,12.84
2957,54292,NaN,Svedka Cucumber Lime,12.0,750,12.38
...,...,...,...,...,...,...
11145,241450,NaN,Cherry Heering,306.0,750,21.99
11146,313902,NaN,Amrut Peated Single Malt Whisky,307.0,750,59.00
11147,379480,NaN,The English Whisky Co. Peated Cask Strength Si...,307.0,750,101.49
11148,120357,NaN,Sullivans Cove Double Cask Single Malt HA,307.0,750,111.26


Las categorias nulas se les colocará la moda de marca_id y el volumen

In [ ]:
producto["categoria_id"] = producto["categoria_id"].fillna(
    producto.groupby(["marca_id", "volumen"])["categoria_id"].transform("mode")
)

In [30]:
len(producto[producto["categoria_id"].isnull()])

16

Debido a que son pocos registros, los cuales no se ha encontrado que tengan una categoria con varias marca y volumen, se les asignara a la categoria "OTHER".

In [31]:
producto.loc[
    producto["categoria_id"].isnull(),
    ["categoria_id"]
] = 102

In [32]:
len(producto[producto["categoria_id"].isnull()])

0

In [33]:
producto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12024 entries, 0 to 12025
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            12024 non-null  int64  
 1   categoria_id  12024 non-null  float64
 2   nombre        12024 non-null  object 
 3   marca_id      12024 non-null  float64
 4   volumen       12024 non-null  int64  
 5   precio        12024 non-null  float64
dtypes: float64(3), int64(2), object(1)
memory usage: 657.6+ KB


Tratados los datos faltantes, se cambiara el tipo de dato

In [34]:
producto["id"] = producto["id"].astype("int")
producto["categoria_id"] = producto["categoria_id"].astype("int")
producto["marca_id"] = producto["marca_id"].astype("int")

In [35]:
producto["nombre"] = producto["nombre"].str.upper()

In [36]:
producto.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12024 entries, 0 to 12025
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            12024 non-null  int32  
 1   categoria_id  12024 non-null  int32  
 2   nombre        12024 non-null  object 
 3   marca_id      12024 non-null  int32  
 4   volumen       12024 non-null  int64  
 5   precio        12024 non-null  float64
dtypes: float64(1), int32(3), int64(1), object(1)
memory usage: 516.7+ KB


### Limpieza clientes

In [37]:
cliente.head()

,id,nombre,apellido,nacimiento,genero,empresa,idioma,nit,puesto,ciudad,correo,telefono
0,599528.0,Samuel,Ward,4/6/89,Male,Yakijo,Marathi,411-44-7088,Geologist IV,Wangjing,sward0@tamu.edu,86-(786)608-5061
1,121688.0,Willie,Gonzales,6/29/72,Male,Zoonoodle,Maltese,701-87-7540,Programmer III,El Corozo,wgonzales1@apache.org,58-(265)301-3397
2,552148.0,Betty,Spencer,9/2/83,Female,Youtags,Dhivehi,373-88-4503,Engineer III,Jinhua,bspencer2@shutterfly.com,86-(195)193-9042
3,102019.0,Beverly,Jordan,1/15/72,Female,Fivespan,Hindi,447-80-5871,Software Test Engineer IV,Salvacion,bjordan3@vimeo.com,63-(652)708-7688
4,189384.0,Cynthia,Flores,2/6/71,Female,Jabbersphere,Tsonga,803-60-8259,Speech Pathologist,Khorol,cflores4@webeden.co.uk,380-(373)389-5435


In [38]:
cliente.isnull().sum()

id            281
nombre        281
apellido      281
nacimiento    281
genero        281
empresa       281
idioma        281
nit           281
puesto        281
ciudad        281
correo        281
telefono      281
dtype: int64

Coinciden muchos datos que sean nulos, primero se vera si hay duplicados en esos nulos

In [39]:
cliente.duplicated().sum()

280

In [40]:
cliente[cliente.duplicated()]

,id,nombre,apellido,nacimiento,genero,empresa,idioma,nit,puesto,ciudad,correo,telefono
11720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11724,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se eliminaran los duplicados

In [41]:
cliente.drop_duplicates(inplace=True)

In [42]:
cliente.duplicated().sum()

0

In [43]:
cliente.isnull().sum()

id            1
nombre        1
apellido      1
nacimiento    1
genero        1
empresa       1
idioma        1
nit           1
puesto        1
ciudad        1
correo        1
telefono      1
dtype: int64

In [44]:
cliente[cliente["id"].isnull()]

,id,nombre,apellido,nacimiento,genero,empresa,idioma,nit,puesto,ciudad,correo,telefono
11719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
cliente.dropna(inplace=True)

Unificacion de columnas de nombre y apellido

In [46]:
cliente["nombre"] = cliente["nombre"]+" "+cliente["apellido"]
cliente.drop(columns=["apellido"], inplace=True)

In [47]:
unique_counts = cliente.nunique()
print("Unique counts per column (using nunique()):")
print(unique_counts)

Unique counts per column (using nunique()):
id            11719
nombre        11475
nacimiento     7547
genero            2
empresa         383
idioma           97
nit           11719
puesto          195
ciudad         9356
correo        11719
telefono      11719
dtype: int64


In [48]:
cliente.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11719 entries, 0 to 11718
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          11719 non-null  float64
 1   nombre      11719 non-null  object 
 2   nacimiento  11719 non-null  object 
 3   genero      11719 non-null  object 
 4   empresa     11719 non-null  object 
 5   idioma      11719 non-null  object 
 6   nit         11719 non-null  object 
 7   puesto      11719 non-null  object 
 8   ciudad      11719 non-null  object 
 9   correo      11719 non-null  object 
 10  telefono    11719 non-null  object 
dtypes: float64(1), object(10)
memory usage: 1.1+ MB


In [49]:
cliente["id"] = cliente["id"].astype("int")

In [ ]:
cliente["nacimiento"] = cliente["nacimiento"].map(parse_mdY)

In [51]:
max(cliente["nacimiento"]), min(cliente["nacimiento"])

(Timestamp('2000-06-30 00:00:00'), Timestamp('1940-05-28 00:00:00'))

In [52]:
cliente["edad"] = datetime.datetime.now().year - cliente["nacimiento"].dt.year
cliente["edad"].describe()

count    11719.000000
mean        42.116990
std         16.462302
min         25.000000
25%         30.000000
50%         36.000000
75%         51.000000
max         85.000000
Name: edad, dtype: float64

In [53]:
cliente.drop(columns=["nit","correo","telefono"], inplace=True)

In [54]:
cliente["nombre"] = cliente["nombre"].str.upper()
cliente["genero"] = cliente["genero"].str.upper()
cliente["genero"] = cliente["genero"].replace({"F": "FEMALE", "M": "MALE"})
cliente["empresa"] = cliente["empresa"].str.upper()
cliente["idioma"] = cliente["idioma"].str.upper()
cliente["puesto"] = cliente["puesto"].str.upper()
cliente["ciudad"] = cliente["ciudad"].str.upper()

### Limpieza events

In [55]:
events.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


In [56]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
 #   Column         Dtype  
---  ------         -----  
 0   timestamp      int64  
 1   visitorid      int64  
 2   event          object 
 3   itemid         int64  
 4   transactionid  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


In [57]:
events["fecha"] = pd.to_datetime(events["timestamp"], unit="ms")
events.drop(columns=["timestamp"], inplace=True)
events.head()

,visitorid,event,itemid,transactionid,fecha
0,257597,view,355908,NaN,2015-06-02 05:02:12.117
1,992329,view,248676,NaN,2015-06-02 05:50:14.164
2,111016,view,318965,NaN,2015-06-02 05:13:19.827
3,483717,view,253185,NaN,2015-06-02 05:12:35.914
4,951259,view,367447,NaN,2015-06-02 05:02:17.106


In [58]:
events.duplicated().sum()

460

In [59]:
events.drop_duplicates(inplace=True)

In [60]:
events.isnull().sum()

visitorid              0
event                  0
itemid                 0
transactionid    2733184
fecha                  0
dtype: int64

In [61]:
events[events["transactionid"].notnull()]["event"].value_counts()

event
transaction    22457
Name: count, dtype: int64

In [62]:
events[events["transactionid"].isnull()]["event"].value_counts()

event
view         2664218
addtocart      68966
Name: count, dtype: int64

Existe un id asignado de transacción únicamente cuando se ha procesado una compra, los demás donde el evento es "view" o "addtocart" no poseen id. Lo mejor será eliminar la columna de transactionid ya que tampoco guarda relación con otras tablas.

In [63]:
min(events["fecha"]), max(events["fecha"])

(Timestamp('2015-05-03 03:00:04.384000'),
 Timestamp('2015-09-18 02:59:47.788000'))

In [64]:
events.drop(columns=["transactionid"], inplace=True)
events.head(3)

,visitorid,event,itemid,fecha
0,257597,view,355908,2015-06-02 05:02:12.117
1,992329,view,248676,2015-06-02 05:50:14.164
2,111016,view,318965,2015-06-02 05:13:19.827


In [65]:
events["event"] = events["event"].str.upper()
events.head()

,visitorid,event,itemid,fecha
0,257597,VIEW,355908,2015-06-02 05:02:12.117
1,992329,VIEW,248676,2015-06-02 05:50:14.164
2,111016,VIEW,318965,2015-06-02 05:13:19.827
3,483717,VIEW,253185,2015-06-02 05:12:35.914
4,951259,VIEW,367447,2015-06-02 05:02:17.106


### Unir a producto el nombre de la categoria y marca

In [66]:
producto = pd.merge(producto, marca, left_on="marca_id", right_on="id", how="left",indicator=True)
producto.head()

,id_x,categoria_id,nombre,marca_id,volumen,precio,id_y,marca,_merge
0,356475,9,CROWN ROYAL HONEY,1,750,22.49,1,DIAGEO AMERICAS,both
1,15335,9,CROWN ROYAL REGAL APPLE MINI,1,300,11.03,1,DIAGEO AMERICAS,both
2,81345,9,CROWN ROYAL REGAL APPLE,1,200,7.08,1,DIAGEO AMERICAS,both
3,150318,9,CROWN ROYAL XR CANADIAN WHISKEY,1,750,98.99,1,DIAGEO AMERICAS,both
4,310791,9,CROWN ROYAL CANADIAN WHISKY MINI,1,300,11.03,1,DIAGEO AMERICAS,both


In [67]:
producto["_merge"].value_counts()

_merge
both          12024
left_only         0
right_only        0
Name: count, dtype: int64

No se han producido datos nulos ya que hay coincidencia en ambas catálogos

In [68]:
producto.drop(columns=["_merge","id_y","marca_id"], inplace=True)

In [69]:
producto = pd.merge(producto, categoria, left_on="categoria_id", right_on="id", how="left",indicator=True)
producto.head()

,id_x,categoria_id,nombre,volumen,precio,marca,id,categoria,_merge
0,356475,9,CROWN ROYAL HONEY,750,22.49,DIAGEO AMERICAS,9,CANADIAN WHISKIES,both
1,15335,9,CROWN ROYAL REGAL APPLE MINI,300,11.03,DIAGEO AMERICAS,9,CANADIAN WHISKIES,both
2,81345,9,CROWN ROYAL REGAL APPLE,200,7.08,DIAGEO AMERICAS,9,CANADIAN WHISKIES,both
3,150318,9,CROWN ROYAL XR CANADIAN WHISKEY,750,98.99,DIAGEO AMERICAS,9,CANADIAN WHISKIES,both
4,310791,9,CROWN ROYAL CANADIAN WHISKY MINI,300,11.03,DIAGEO AMERICAS,9,CANADIAN WHISKIES,both


In [70]:
producto["_merge"].value_counts()

_merge
both          12024
left_only         0
right_only        0
Name: count, dtype: int64

In [71]:
producto.drop(columns=["_merge","id","categoria_id"], inplace=True)
producto.rename(columns={"id_x": "id"}, inplace=True)
producto.head()

,id,nombre,volumen,precio,marca,categoria
0,356475,CROWN ROYAL HONEY,750,22.49,DIAGEO AMERICAS,CANADIAN WHISKIES
1,15335,CROWN ROYAL REGAL APPLE MINI,300,11.03,DIAGEO AMERICAS,CANADIAN WHISKIES
2,81345,CROWN ROYAL REGAL APPLE,200,7.08,DIAGEO AMERICAS,CANADIAN WHISKIES
3,150318,CROWN ROYAL XR CANADIAN WHISKEY,750,98.99,DIAGEO AMERICAS,CANADIAN WHISKIES
4,310791,CROWN ROYAL CANADIAN WHISKY MINI,300,11.03,DIAGEO AMERICAS,CANADIAN WHISKIES


### Unir events con producto

In [72]:
events1 = pd.merge(events, producto, left_on="itemid", right_on="id", how="left", indicator=True)
events1.head()

,visitorid,event,itemid,fecha,id,nombre,volumen,precio,marca,categoria,_merge
0,257597,VIEW,355908,2015-06-02 05:02:12.117,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,992329,VIEW,248676,2015-06-02 05:50:14.164,248676.0,MARGARITAVILLE TEQUILA ORO(GOLD),1000.0,11.75,"SAZERAC CO., INC.",TEQUILA,both
2,111016,VIEW,318965,2015-06-02 05:13:19.827,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,483717,VIEW,253185,2015-06-02 05:12:35.914,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,951259,VIEW,367447,2015-06-02 05:02:17.106,367447.0,THE DALMORE CIGAR MALT SCOTCH,750.0,31.49,E AND J GALLO WINERY,SINGLE MALT SCOTCH,both


In [73]:
events1.drop(columns=["id"], inplace=True)

In [74]:
events1["_merge"].value_counts()    

_merge
left_only     1841882
both           913916
right_only          0
Name: count, dtype: int64

Los productos que aparecen en left only se podría interpretar como productos que han sido descontinuados dentro de la base de datos.

### Events con clientes

In [75]:
events2 = pd.merge(events, cliente, left_on="visitorid", right_on="id", how="left", indicator=True)
events2.head()

,visitorid,event,itemid,fecha,id,nombre,nacimiento,genero,empresa,idioma,puesto,ciudad,edad,_merge
0,257597,VIEW,355908,2015-06-02 05:02:12.117,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,992329,VIEW,248676,2015-06-02 05:50:14.164,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,111016,VIEW,318965,2015-06-02 05:13:19.827,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,483717,VIEW,253185,2015-06-02 05:12:35.914,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,951259,VIEW,367447,2015-06-02 05:02:17.106,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [76]:
events2.drop(columns=["id"], inplace=True)

In [77]:
events2["_merge"].value_counts()

_merge
left_only     2525035
both           230606
right_only          0
Name: count, dtype: int64

El left only indica que son clientes que no se han registrado en nuestro sistema de datos y son ids temporales

In [78]:
events2[events2["_merge"] == "left_only"]["event"].value_counts()

event
VIEW         2483475
ADDTOCART      41560
Name: count, dtype: int64

Estos datos corresponden a aquellos usuarios que solamente se encuentran curiosos por los productos ofrecidos pero no concluyen la compra.

### Unir eventos1 con eventos2

In [79]:
events1.columns, events2.columns

(Index(['visitorid', 'event', 'itemid', 'fecha', 'nombre', 'volumen', 'precio',
        'marca', 'categoria', '_merge'],
       dtype='object'),
 Index(['visitorid', 'event', 'itemid', 'fecha', 'nombre', 'nacimiento',
        'genero', 'empresa', 'idioma', 'puesto', 'ciudad', 'edad', '_merge'],
       dtype='object'))

In [80]:
events1 = events1.rename(columns={"_merge": "event+producto"})
events2 = events2.rename(columns={"_merge": "event+cliente"})

In [81]:
cols_to_use = [c for c in events2.columns if c not in events1.columns or c in ["visitorid", "itemid"]]

eventos3 = pd.merge(
    events1,
    events2[cols_to_use],
    on=["visitorid", "itemid"],
    how="outer"
)

In [82]:
eventos3.head()

,visitorid,event,itemid,fecha,nombre,volumen,precio,marca,categoria,event+producto,nacimiento,genero,empresa,idioma,puesto,ciudad,edad,event+cliente
0,257597,VIEW,355908,2015-06-02 05:02:12.117,NaN,NaN,NaN,NaN,NaN,left_only,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
1,992329,VIEW,248676,2015-06-02 05:50:14.164,MARGARITAVILLE TEQUILA ORO(GOLD),1000.0,11.75,"SAZERAC CO., INC.",TEQUILA,both,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
2,111016,VIEW,318965,2015-06-02 05:13:19.827,NaN,NaN,NaN,NaN,NaN,left_only,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
3,111016,VIEW,318965,2015-06-02 05:13:19.827,NaN,NaN,NaN,NaN,NaN,left_only,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,111016,VIEW,318965,2015-05-31 02:27:57.492,NaN,NaN,NaN,NaN,NaN,left_only,NaT,NaN,NaN,NaN,NaN,NaN,NaN,left_only


In [83]:
eventos3[["event+producto","event+cliente"]].value_counts()

event+producto  event+cliente
left_only       left_only        3576266
both            left_only        1995998
                both              947358
left_only       both              273810
Name: count, dtype: int64

| event+producto | event+cliente | count     | Interpretación                                                                                            |
| -------------- | ------------- | --------- | --------------------------------------------------------------------------------------------------------- |
| left\_only     | left\_only    | 3,576,266 | Evento que no encontró coincidencia ni en producto ni en cliente. Son eventos “huérfanos” en ambos joins. |
| both           | left\_only    | 1,995,998 | Evento que sí tiene producto asociado pero no cliente asociado.                                           |
| both           | both          | 947,358   | Evento que tiene tanto producto como cliente asociados (join completo en ambas).                          |
| left\_only     | both          | 273,810   | Evento que no tiene producto asociado pero sí cliente.                                                    |


Eliminacion de usuarios con edades negativas o superior a 110 

### Exportar los datos

In [84]:
os.makedirs("processed_data", exist_ok=True)
cliente.to_csv("processed_data/cliente.csv", index=False)
categoria.to_csv("processed_data/categoria.csv", index=False)
marca.to_csv("processed_data/marca.csv", index=False)
producto.to_csv("processed_data/producto.csv", index=False)
events.to_csv("processed_data/evento.csv", index=False)

events1.to_csv("processed_data/eventos_productos.csv", index=False)
events2.to_csv("processed_data/eventos_clientes.csv", index=False)
eventos3.to_csv("processed_data/events_productos_clientes.csv", index=False)